<a href="https://colab.research.google.com/github/GleiconMaior/GleiconMaior/blob/main/Dimension_stones_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import Delaunay
from matplotlib.path import Path

def load_data(file_path):
    """
    Carrega os dados do arquivo de pontos do sólido e renomeia as colunas.
    """
    try:
        df = pd.read_csv(file_path, sep=None, engine="python", header=0)
        col_mapping = {'*X-Coordinate': 'X', '*Y-Coordinate': 'Y', '*Z-Coordinate': 'Z'}
        df.rename(columns=col_mapping, inplace=True)

        if {'X', 'Y', 'Z'}.issubset(df.columns):
            df.dropna(inplace=True)
            return df
        else:
            print(f"Erro: Colunas detectadas {df.columns.tolist()}, esperado: ['X', 'Y', 'Z']")
            return pd.DataFrame(columns=['X', 'Y', 'Z'])
    except Exception as e:
        print(f"Erro ao carregar {file_path}: {e}")
        return pd.DataFrame(columns=['X', 'Y', 'Z'])

def calculate_removed_volume(df_original, vertices, zmin):
    """
    Calcula o volume do bloco removido com base nos pontos originais dentro do polígono e usando zmin como base.

    Parâmetros:
    - df_original: DataFrame com os pontos originais antes do corte.
    - vertices: Lista com os vértices do polígono de corte.
    - zmin: Cota da base do polígono para cálculo do volume.

    Retorna:
    - volume_total: Volume do bloco removido.
    """

    # Criar polígono da área removida
    polygon = Path(vertices)

    # Filtrar pontos dentro do polígono e acima do zmin (área removida)
    points = df_original[['X', 'Y']].to_numpy()
    inside_polygon = polygon.contains_points(points)

    df_removed = df_original[inside_polygon & (df_original['Z'] >= zmin)].copy()

    if df_removed.empty:
        print("Nenhum ponto foi removido. O volume é zero.")
        return 0

    # Criar triangulação Delaunay para os pontos removidos
    xy_points = df_removed[['X', 'Y']].to_numpy()
    tri = Delaunay(xy_points)

    # Calcular volume do bloco removido
    volume_total = 0

    for simplex in tri.simplices:
        # Pegar os vértices do triângulo
        p1, p2, p3 = xy_points[simplex]

        # Obter as cotas Z dos pontos originais
        z1, z2, z3 = df_removed.loc[df_removed[['X', 'Y']].apply(tuple, axis=1).isin([tuple(p1), tuple(p2), tuple(p3)]), 'Z'].to_numpy()

        # Volume do prisma triangular: (Área da base) * (Altura média)
        area_base = 0.5 * abs((p2[0] - p1[0]) * (p3[1] - p1[1]) - (p3[0] - p1[0]) * (p2[1] - p1[1]))
        altura_media = ((z1 + z2 + z3) / 3) - zmin

        volume = area_base * altura_media
        volume_total += volume

    return volume_total

# Caminho do arquivo original antes da remoção
solid_path = "/content/solido.txt"
df_solid_original = load_data(solid_path)

# Perguntar ao usuário as coordenadas dos vértices do polígono de corte
num_vertices = int(input("Quantos vértices o polígono terá? (Ex: 4 para um retângulo, mais para polígonos complexos): "))
vertices = []

print("Digite as coordenadas X e Y dos vértices do polígono de corte:")
for i in range(num_vertices):
    x = float(input(f"Digite o valor de X do vértice {i+1}: "))
    y = float(input(f"Digite o valor de Y do vértice {i+1}: "))
    vertices.append((x, y))

# Perguntar ao usuário o valor da cota mínima Z (base do bloco removido)
zmin = float(input("Digite o valor de Z mínimo (cota da base do bloco removido): "))

# Calcular volume
volume_removido = calculate_removed_volume(df_solid_original, vertices, zmin)
print(f"Volume do bloco removido: {volume_removido:.2f} m³")